In [1]:
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent

In [2]:
from langchain_tavily import TavilySearch
tool = TavilySearch(max_results=2)

In [3]:
max_iterations = 3
recursion_limit = 2 * max_iterations + 1
agent = create_react_agent(
    model="anthropic:claude-3-5-haiku-latest",
    tools=[tool]
)
agent_with_recursion_limit = agent.with_config(recursion_limit=recursion_limit)

try:
    response = agent_with_recursion_limit.invoke(
        {"messages": [{"role": "user", "content": "Tell me about the latest news in India Pakistan War"}]},
    )
except GraphRecursionError:
    print("Agent stopped due to max iterations.")

In [4]:
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on the search results, here's the latest news about India-Pakistan tensions:

1. Recent Conflict Escalation:
- India launched "Operation Sindoor," a military operation targeting what it calls "terrorist infrastructure" in Pakistan and Pakistan-administered Kashmir.
- This operation was in retaliation to a deadly militant attack in Indian-administered Kashmir on April 22nd that killed at least 26 people.

2. Diplomatic Tensions:
- India has initiated a diplomatic outreach to countries like the U.S., Japan, and China following the military strikes.
- The country has suspended a water treaty with Pakistan and closed border crossings.

3. Context of Conflict:
- The tensions are rooted in the long-standing dispute over Kashmir, a region both countries have claimed since their partition in 1947.
- This is not a new conflict, as India and Pakistan have fought multiple wars over Kashmir since 1947, includin

Streaming messages 

There are multiple stream modes - updates will give each message of state messages as output.

In [6]:
agent = create_react_agent(
    model="anthropic:claude-3-5-haiku-latest",
    tools=[tool],
)
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Recent news in India Pakistan War"}]},
    stream_mode="updates"
):
    print(chunk)
    print("\n")

{'agent': {'messages': [AIMessage(content=[{'text': 'Let me search for recent news about tensions between India and Pakistan.', 'type': 'text'}, {'id': 'toolu_01EuCViKxCvkgKpftEJFqdHj', 'input': {'query': 'Recent India Pakistan tensions or conflict 2024', 'topic': 'news', 'time_range': 'month'}, 'name': 'tavily_search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_014HgBm56PKLr5jbv1NGaMh7', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1378, 'output_tokens': 112, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run--4d18d2ea-1276-4474-8987-b149dc7cba64-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Recent India Pakistan tensions or conflict 2024', 'topic': 'news', 'time_range': 'month'}, 'id': 'toolu_01EuCViKxCvkgKpftEJFqdHj', 'type': 'tool_call'}], usage_metada

Above is similar to verbose what the agent does

Below is like streaming the output of LLMs.

In [6]:
agent = create_react_agent(
    # model="openai:gpt-4.1-mini",
    model="anthropic:claude-3-5-haiku-latest",
    tools=[tool],
)
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "what is the latest news in India Pakistan War"}]},
    stream_mode="messages"  # Changed from "messages" to "updates"
):
    print(chunk)
    # print("\n")

# TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'
# During task with name 'agent' and id '79a68202-0769-ad08-e500-1b7e6339f58d'

# I am getting this error when I run the code above
# Error went of when I upgraded langchain anthropic to latest version, 0.3.13

(AIMessageChunk(content=[], additional_kwargs={}, response_metadata={'model_name': 'claude-3-5-haiku-20241022'}, id='run--f440d5b2-88bc-40fb-8e2b-fa058376c49a', usage_metadata={'input_tokens': 1381, 'output_tokens': 0, 'total_tokens': 1381, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:3e40c1f8-41b2-9fef-962b-169f666fec0a', 'checkpoint_ns': 'agent:3e40c1f8-41b2-9fef-962b-169f666fec0a', 'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-5-haiku-latest', 'ls_model_type': 'chat', 'ls_temperature': None, 'ls_max_tokens': 1024})
(AIMessageChunk(content=[{'id': 'toolu_0146gjyYSVUqc9E1SxbtGT3V', 'input': {}, 'name': 'tavily_search', 'type': 'tool_use', 'index': 0}], additional_kwargs={}, response_metadata={}, id='run--f440d5b2-88bc-40fb-8e2b-fa058376c49a', tool_calls=[{'name': 'tavily_search', 'args'

Adding model fallbacks

Often times we ask models to output format in a specific format (like JSON). Models like GPT-3.5 can do this okay, but sometimes struggle. This naturally points to fallbacks - we can try with GPT-3.5 (faster, cheaper), but then if parsing fails we can use GPT-4.

In [7]:
from langchain.chat_models import init_chat_model

model_with_fallbacks = (
    init_chat_model("anthropic:claude-3-5-haiku-latest")
    .with_fallbacks([
        init_chat_model("openai:gpt-4.1-mini"),
    ])
)

Above will act as a fallback for LLMs, but we know that 3.5 haiku is a more capable model than 4.1 mini. So we can use 4.1-mini and use 3.5 haiku as fallback - above is just an illustration